## Step 1 Install dependencies and enviornment

In [1]:
!pip install -q tensorflow-gpu==2.0.0-beta1
!pip install pandas-datareader

## Step 2 Import Dependencies

In [0]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

In [3]:
tf.__version__

'2.0.0-beta1'

## Step 3 building the AI Trader network

In [0]:
class AI_Trader():
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Stay, Buy, Sell
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name
    
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    
    self.model = self.model_builder()
    
  def model_builder(self):
    
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))
    
    return model
  
  def trade(self, state):
    
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  
  def batch_train(self, batch_size):
    
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

## Step 4 Dataset Preprocessing

### Sigmoid

In [0]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

### Price formal function

In [0]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

### Dataset loader

In [0]:
def dataset_loader(stock_name):
  
  #complete the dataset loader function
  dataset = data_reader.DataReader(stock_name,data_source = "yahoo")
  
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  
  close = dataset['Close']
  
  return close

### State Creator

In [0]:
def state_creator(data, timestep, window_size):
  starting_id = timestep - window_size +1 
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] +list(data[0:timestep+1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
  
  return np.array([state])

### Loading a dataset 

In [0]:
stock_name = "AAPL"
data = dataset_loader(stock_name)

## Step 5 Training the AI Trader

### Setting hyper parameters

In [0]:
window_size = 10
episodes = 1000

batch_size = 32
data_samples = len(data) - 1

### Defining the Trader model

In [0]:
trader = AI_Trader(window_size)

In [12]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


### Training Loop

In [0]:
for episode in range(1, episodes + 1):
  print(f"Episode: {episode}/{episodes}")
  
  state = state_creator(data, 0 , window_size + 1)
  
  total_profit = 0 
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0
    
    if action == 1: # buying
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))
    
    elif action ==2 and len(trader.inventory) > 0: #selling
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("######################")
      print(f"TOTAL PROFIT: {total_profit}")
      print("######################")
      
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episode % 10 == 0:
    trader.model.save(f"ai_trader_{episode}.h5")

  0%|          | 0/2416 [00:00<?, ?it/s]

Episode: 1/1000
AI Trader bought:  $ 30.282858
AI Trader bought:  $ 30.015715
AI Trader bought:  $ 29.674286
AI Trader sold:  $ 30.092857  Profit: - $ 0.190001
AI Trader bought:  $ 29.918571
AI Trader sold:  $ 29.418571  Profit: - $ 0.597143
AI Trader sold:  $ 30.247143  Profit: $ 0.572857
AI Trader bought:  $ 29.724285
AI Trader bought:  $ 29.010000
AI Trader bought:  $ 29.420000
AI Trader bought:  $ 29.697144
AI Trader sold:  $ 27.437143  Profit: - $ 2.481428
AI Trader sold:  $ 27.818571  Profit: - $ 1.905714
AI Trader sold:  $ 27.980000  Profit: - $ 1.030001
AI Trader bought:  $ 28.461428
AI Trader sold:  $ 27.435715  Profit: - $ 1.984285
AI Trader bought:  $ 27.922857


  1%|          | 25/2416 [00:01<03:01, 13.18it/s]

AI Trader sold:  $ 27.731428  Profit: - $ 1.965715
AI Trader sold:  $ 28.027143  Profit: - $ 0.434284


  1%|          | 28/2416 [00:01<05:27,  7.28it/s]

...

 11%|█▏        | 275/2416 [00:42<05:59,  5.95it/s]

AI Trader bought:  $ 49.062859
AI Trader bought:  $ 49.500000


 11%|█▏        | 277/2416 [00:42<05:56,  5.99it/s]

AI Trader sold:  $ 50.268570  Profit: $ 1.205711
AI Trader sold:  $ 50.742859  Profit: $ 1.242859


 12%|█▏        | 279/2416 [00:43<05:58,  5.96it/s]